In [ ]:
################# Dependencies

In [ ]:
import os
import openai
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index import download_loader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext, LLMPredictor
import datetime
from llama_index import StorageContext, load_index_from_storage
from llama_index import GPTVectorStoreIndex
import json
import os
import glob
JsonDataReader = download_loader("JsonDataReader")

In [ ]:
####################Variables, objects creation

In [ ]:
# Setting parameters for GPT3.5 model
openai.api_key = os.environ['OPENAI_API_KEY']
gpt_35_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3))
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = SimpleInputPrompt(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST]"
)

In [ ]:
###################  Data loading and index creation

In [ ]:
# Data loading
# JSON reader
JsonDataReader = download_loader("JsonDataReader")
# path to the data
path = '/notebooks/Data/Jsondata/'
docs = []
count = 0
for filename in glob.glob(os.path.join(path, '*.json')): 
    with open(filename, encoding='utf-8', mode='r') as currentFile:
        data=currentFile.read()
        docs.append(data)
        print(str(count))
        count = count + 1
loader = JsonDataReader()
documents = loader.load_data(docs)
# Creating indexing

index = GPTVectorStoreIndex.from_documents(documents, service_context=gpt_35_context)
#storing indexing
index.storage_context.persist("/notebooks/data/indexes/GPT_Index")